In [1]:
# modified from https://github.com/uci-cbcl/FactorNet/blob/master/visualize.py
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]=""

import gzip
import re
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from keras import Model
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import RMSprop
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D,Dense,Dropout,TimeDistributed,Flatten,BatchNormalization,Bidirectional,LSTM
from keras.models import model_from_json
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
model = keras.models.load_model('ES_model.h5')
print (model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 140, 4)       0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 140, 4)       0                                            
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 134, 32)      928         input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 134, 32)      928         input_1[0][0]                    
__________________________________________________________________________________________________
dropout_4 

In [3]:
x1=np.load('x1.npy')
x2=np.load('x2.npy')

In [4]:
x1.shape

(5090, 140, 4)

In [5]:
x2.shape

(5090, 140, 4)

In [6]:
model.inputs

[<tf.Tensor 'input_1:0' shape=(?, 140, 4) dtype=float32>,
 <tf.Tensor 'input_2:0' shape=(?, 140, 4) dtype=float32>,
 <tf.Tensor 'input_3:0' shape=(?, 3) dtype=float32>]

In [7]:
 model.layers[2]

In [8]:
conv_layer1  = model.layers[3]
conv_layer2  = model.layers[2]


In [9]:
conv_layer1

In [10]:
model.inputs[0]

<tf.Tensor 'input_1:0' shape=(?, 140, 4) dtype=float32>

In [55]:
model.inputs[1]

<tf.Tensor 'input_2:0' shape=(?, 140, 4) dtype=float32>

In [54]:
conv_layer1.input

<tf.Tensor 'input_1:0' shape=(?, 140, 4) dtype=float32>

In [11]:
conv_layer2.input

<tf.Tensor 'input_2:0' shape=(?, 140, 4) dtype=float32>

In [56]:
f=K.function([model.inputs[0], model.inputs[1]],[K.argmax(tf.maximum(conv_layer1.output,0),axis=1), K.max(tf.maximum(conv_layer1.output,0),axis=1),K.argmax(tf.maximum(conv_layer2.output,0),axis=1), K.max(tf.maximum(conv_layer2.output,0),axis=1)])
z = f([x1,x2])

In [57]:
z[1].shape

(5090, 32)

In [58]:
max_inds = z[0] # N x M matrix, where M is the number of motifs
max_acts = z[1]
max_inds_rc = z[2]
max_acts_rc = z[3]


In [59]:
K.function

<function keras.backend.tensorflow_backend.function(inputs, outputs, updates=None, **kwargs)>

In [60]:
np.min(max_acts)

0.0

In [61]:
num_motifs = 32
w=7
motifs = np.zeros((num_motifs, w, 4))
nsites = np.zeros(num_motifs)

for m in range(num_motifs):
    for n in range(len(x1[0])):
        # Forward strand
        if max_acts[n, m] > 0:
            nsites[m] += 1
            motifs[m] += x1[n, max_inds[n, m]:max_inds[n, m] + w, :]



In [62]:
f = open('motifs_ES1.txt', 'w')
f.write('MEME version 4.9.0\n\n'
        'ALPHABET= ACGT\n\n'
        'strands: + -\n\n'
        'Background letter frequencies (from uniform background):\n'
        'A 0.25000 C 0.25000 G 0.25000 T 0.25000\n\n')
for m in range(num_motifs):
    if nsites[m] == 0:
        continue
    f.write('MOTIF M%i O%i\n' % (m, m))
    f.write("letter-probability matrix: alength= 4 w= %i nsites= %i E= 1337.0e-6\n" % (w, nsites[m]))
    for j in range(w):
        f.write("%f %f %f %f\n" % tuple(1.0 * motifs[m, j, 0:4] / np.sum(motifs[m, j, 0:4])))
    f.write('\n')
f.close()

In [63]:
num_motifs = 32
w=7
motifs = np.zeros((num_motifs, w, 4))
nsites = np.zeros(num_motifs)

for m in range(num_motifs):
    for n in range(len(x1[0])):

        # Reverse strand
        if max_acts_rc[n, m] > 0:
            nsites[m] += 1
            motifs[m] += x2[n, max_inds_rc[n, m]:max_inds_rc[n, m] + w, :] 

In [66]:
f = open('motifs_ES2.txt', 'w')
f.write('MEME version 4.9.0\n\n'
        'ALPHABET= ACGT\n\n'
        'strands: + -\n\n'
        'Background letter frequencies (from uniform background):\n'
        'A 0.25000 C 0.25000 G 0.25000 T 0.25000\n\n')
for m in range(num_motifs):
    if nsites[m] == 0:
        continue
    f.write('MOTIF M%i 0%i\n' % (m+32, m+32))
    f.write("letter-probability matrix: alength= 4 w= %i nsites= %i E= 1337.0e-6\n" % (w, nsites[m]))
    for j in range(w):
        f.write("%f %f %f %f\n" % tuple(1.0 * motifs[m, j, 0:4] / np.sum(motifs[m, j, 0:4])))
    f.write('\n')
f.close()

In [158]:
output= ''
os.system('tomtom -dist pearson -thresh 0.05 -eps -oc %s/tomtom motifs_cas %s' % (output, 'motifdb.meme'))

32512

In [ ]:
tomtom